<img src="./images/DLI_Header.png" style="width: 400px;">

# 3. NVIDIA FasterTransformer를 사용한 GPT-J 6b 추론


이번 실습에서는 NVIDIA FasterTransformer 라이브러리를 살펴보고, 이 라이브러리의 작동 방식 및 모델에 적용되는 최적화 기법에 대해 설명하겠습니다. 
이후, NVIDIA FasterTransformers를 사용하여 6b의 매개 변수를 가진 큰 GPT-J 모델의 추론을 실행하는 방법에 대한 기본 예제를 실행해 봅니다.
그리고, 텐서 병렬(TP) 및/또는 파이프라인 병렬(PP)과 같은 다양한 추론 모드에 대해 살펴보겠습니다. 또한 HuggingFace와 FasterTransformer TP/PP 추론 접근법 간의 지연 시간을 비교할 것입니다.

## 목표

이번 노트북의 목표는 다음과 같습니다:
* NVIDIA FasterTransformer 라이브러리가 무엇인지 알아봅니다
* 이 라이브러리를 구축하는 방법에 대해 알아봅니다
* FasterTransformer를 사용하여 추론을 위해 Transformer 모델의 체크포인트를 준비하는 방법에 대해 알아봅니다
* 하나의 GPU에서 모델의 추론을 실행합니다
* 텐서 병렬화 기법을 사용하여 여러 GPU에서 모델 추론을 실행합니다
* 지연 시간을 확인하기 위해 모든 파이프라인을 프로파일링합니다.
* 추론 속도를 측정하고 프로파일링 결과를 분석합니다.


**[3.1 NVIDIA FasterTransformer](#3.1)<br>**
**[3.2 NVIDIA FasterTransformer 전체적인 추론 파이프라인](#3.2)<br>**
**[3.3 NVIDIA FasterTransformer 라이브러리 다운로드 및 구축하기](#3.3)<br>**
**[3.4 GPT-J 체크포인트 다운로드 및 준비하기](#3.4)<br>**
**[3.5 추론을 위해 FT-친화적인 포맷으로 가중치 변환하기](#3.5)<br>**
**[3.6 C++ 바인딩 활용 GPT-J 추론](#3.6)<br>**
&nbsp;&nbsp;&nbsp;&nbsp;[3.6.1 1-GPU 추론 ](#3.4.1)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[3.6.2 2-GPU 추론 ](#3.4.2)<br>

# 3.1 NVIDIA FasterTransformer

## FasterTransformer 개요

[NVIDIA FasterTransformer (FT)](https://github.com/NVIDIA/FasterTransformer) 는 분산 방식으로 거대 모델의 추론에 중점을 둔 트랜스포머 기반 신경망 추론을 위한 가속 엔진을 구현하는 라이브러리입니다. 
FasterTransformer에는 인코더 및 디코더 부품이 포함된 고도로 최적화된 버전의 Transformer 블록이 구현되어 있습니다. 이 블록을 사용하여 T5와 같은 전체 인코더-디코더 아키텍처를 가진 파이프라인으로 추론을 실행할 수 있을 뿐만 아니라 BERT-Like(인코더 전용) 또는 GPT-3 Like(디코더 전용)한 모델의 추론을 별도로 수행할 수 있습니다. C++/CUDA로 작성되었으며 GPU에서 가장 빠른 계산 파이프라인을 구축할 수 있는 cuBLAS, cuBLASlt​, cuSPARSELt 라이브러리에 의존합니다.

## GPT 모델 소개
GPT와 GPT-J는 인코더 모듈이 없고 멀티 헤드 어텐션을 교차하며 액티베이션으로 GeLU를 사용하는 디코딩 모델의 변형입니다. 2020년에 OpenAI는 [자체 논문](https://arxiv.org/abs/2005.14165)에서 거대 모델과 많은 훈련 데이터를 사용하면 GPT 모델의 용량을 크게 향상시킬 수 있음을 보여주었습니다. 예를 들어, 가장 큰 모델인 GPT-3에는 1750억 개의 매개 변수가 있으며, 이는 반 정밀도로 약 350GB가 소요됩니다. 그러나 이러한 모델을 단일 GPU에 넣는 것은 불가능합니다. 따라서 멀티-GPU, 심지어 멀티-노드도 필요합니다. 모델 크기로 인한 지연 시간 및 메모리의 병목 현상을 해결하기 위해 FasterTransformer는 여러 프레임워크에서 높은 효율성, 최적화된 메모리 사용 및 모델 병렬화를 제공하는 커널을 제공합니다. 

이 문서에서는 FasterTransformer가 GPT-J 모델에 제공하는 기능과 워크플로우 및 최적화를 설명합니다. 또한 사용자가 FasterTransformer에서 GPT-J 모델을 실행할 수 있도록 지원하는 가이드를 제공합니다. 

### FasterTransformer - Supported features:

* 체크포인트 컨버터
  * Huggingface
  * Megatron
  * Nemo Megatron
  * TensorFlow
* 데이터 타입
  * FP32
  * FP16
  * INT8 weight only PTQ for bs 1 and 2
* 기능
  * 멀티-GPU 멀티-노드 추론
  * 다이내믹 랜덤 시드 (Dynamic random seed)
  * 토큰 중지 (Stop tokens)
  * 빔 검색 및 샘플링 (Beam search and sampling)
  * FP32, FP16와 INT8 추론
* 프레임워크
  * TensorFlow
  * PyTorch
  * C++
  * Triton 백엔드

### 텐서-병렬화 및 파이프라인-병렬화

FasterTransformer는 MPI 및 NVIDIA NCCL을 사용하여 노드 내/노드 간 통신을 지원합니다. 이 소프트웨어 스택을 사용하면 누구나 여러 GPU에서 텐서-병렬 모드로 거대 트랜스포머를 실행하여 계산 지연 시간을 줄일 수 있습니다. 동시에, 텐서 병렬화(TP)와 파이프라인 병렬화(PP)가 함께 결합되어 멀티 GPU 및 멀티 노드 환경에서 수십억 및 수조 개의 매개 변수(테라바이트의 가중치에 해당하는)를 가진 거대한 트랜스포머 모델을 실행할 수 있습니다. 

- DataParallel (DP) - 동일한 설정이 여러 번 복제되며, 각 설정은 데이터의 슬라이스를 제공합니다. 처리는 병렬로 수행되며 모든 설정은 각 훈련 단계가 끝날 때 동기화됩니다.
- TensorParallel (TP) - 각 텐서는 여러 개의 청크로 분할되므로 전체 텐서가 단일 GPU에 상주하는 대신 텐서의 각 샤드가 지정된 GPU에 상주합니다. 처리하는 동안 각 샤드는 서로 다른 GPU에서 별도로 병렬로 처리되고 결과는 단계가 끝날 때 동기화됩니다. 이것은 수평 레벨에서 분리가 일어나기 때문에 수평 평행(horizontal parallelism)라고 부르기도 합니다.
- Pipeline Parallel (PP) - 모델은 여러 GPU에 걸쳐 수직(레이어 레벨)으로 분할되므로 모델의 한 개 또는 여러 계층만 단일 GPU에 배치됩니다. 각 GPU는 파이프라인의 서로 다른 단계에서 병렬 처리되며 배치의 작은 청크에서 작동합니다.

<img src="./images/image3.png" style="width: 1000px;">

### FasterTransformer 라이브러리의 추론 최적화

FT를 사용하면 딥 러닝 훈련을 위한 일반적인 프레임워크에 비해 트랜스포머 기반 NN에 대한 대기 시간이 짧고 처리량이 높아 더 빠른 추론 파이프라인을 얻을 수 있습니다. 다음은 FT가 GPT-3 메가트론 모델에 대해 가장 빠른 추론을 할 수 있도록 하는 몇 가지 최적화 기술을 소개합니다:
1. <b>Layer Fusion</b></br>
사전 처리 단계에 들어가 하나의 커널로 계산될 수 있도록 여러 층의 NN을 하나로 결합하는 일련의 기술입니다. 이 기술은 데이터 전송을 줄이고 연산적인 밀도를 높여서 추론 단계에서 계산을 가속화합니다. 예를 들어 멀티 헤드 어텐션 블록의 모든 작업을 하나의 커널로 결합할 수 있습니다.
2. <b>Keys/Values caching - 자기 회귀 모델에 대한 추론 최적화</b></br>
FT는 이전 키와 밸류 값을 재계산하지 않도록 각 단계에서 저장할 버퍼를 할당합니다. 추가 메모리 사용량이 필요하지만 FT는 재계산, 각 단계에서 버퍼 할당 및 연결 비용을 절약할 수 있습니다.<br/>
<div style="text-align:center">
<img src="./images/KV_caching v2.PNG" style="width: 60%;position:relative;"><br/>
<em>Keys/Values caching</em>
</div>
<br/><br/>
3. <b> 메모리 최적화 </b> </br>
BERT와 같은 기존 모델과 달리 거대 트랜스포머 모델은 최대 수조 개의 매개 변수를 가집니다. 예를 들어 GPT-3 175b는 모델을 반 정밀도로 저장해도 350GB를 사용합니다. 따라서 우리는 다른 부분의 메모리 사용량을 줄여야 합니다. FasterTransformer에서는 서로 다른 디코더 계층의 메모리 버퍼를 재사용합니다. GPT-3의 레이어 수가 96개이기 때문에, 우리는 1/96 메모리만 있으면 됩니다.
4. <b>노드 내/노드 간 통신을 지원하고 모델 병렬화를 지원하는 MPI 및 NCCL 활용</b></br>
GPT 모델을 사용 시, FasterTransormer는 텐서와 파이프라인 병렬화를 모두 제공합니다. 텐서 병렬화를 위해, FasterTransformer는 Megatron의 아이디어를 따릅니다. 셀프 어텐션블록과 피드 포워드 네트워크 블록에서 FT는 첫 번째 행렬 곱의 가중치를 행으로 분할하고 두 번째 행렬 곱의 가중치를 열로 분할합니다. FT는 최적화를 통해 각 트랜스포머 블록에 대해 감소 연산을 2배까지 줄일 수 있습니다. 파이프라인 병렬화를 위해서FasterTransformer는 전체 요청 배치를 여러 개의 마이크로 배치로 분할하고 통신 버블들을 숨깁니다. FasterTransformer는 상황에 따라 마이크로 배치 크기를 자동으로 조정합니다. 사용자는 gpt_config.ini 파일을 수정하여 모델 병렬화를 조정할 수 있습니다. 텐서 병렬은 더 많은 NCCL 통신이 필요하기 때문에 텐서 병렬 인트라노드와 파이프라인 병렬 인터노드를 사용하는 것을 추천드립니다.
5. <b>MatMul 커널 오토 튜닝 (GEMM 자동 조정) </b> </br>
행렬 곱은 트랜스포머 기반 신경망에서 가장 중요하고 무거운 연산입니다. FT는 CuBLAS 및 CuTLASS 라이브러리의 기능을 사용하여 이러한 유형의 연산을 실행합니다. MatMul 연산은 "하드웨어" 수준에서 서로 다른 로우레벨 알고리즘을 사용하여 수십 가지 다른 방식으로 실행될 수 있다는 것을 아는 것이 중요합니다. GemmBatchedEx 함수는 MatMul 연산을 구현하고  “cublasGemmAlgo_t” 를 입력 매개 변수로 사용합니다. 이 매개 변수를 사용하여, 우리는 작동을 위해 다른 로우 레벨 알고리즘을 선택할 수 있습니다. FasterTransformer 라이브러리는 이 매개 변수를 사용하여 모든 로우 레벨 알고리즘에 대한 실시간 벤치마크를 수행하고 모델의 매개 변수(어텐션 레이어,크기, 어텐션 헤드 수, 히든 레이어 크기 등)와 입력 데이터에 가장 적합한 매개 변수를 선택합니다. 또한 FT는 __expf(), __shfl_xor_sync() 와 같은 일부 네트워크 부분에 하드웨어 가속 로우 레벨 함수를 사용합니다.
<div style="text-align:center">
<img src="./images/kernel autotuning.png" style="width: 20%;position:relative;"><br/>
<em>Kernel autotuning</em>
</div>
<br/><br/>
6. <b>더 낮은 정밀도로 추론</b></br>
FP16, FP32, POC에서 GPT-3에 대한 포스트-트레이닝 가중치 전용 INT8 양자화
FT에는 fp16 및 int8 의 낮은 정밀도 입력 데이터를 사용하여 추론을 지원하는 커널이 있습니다. 이 두 체제 모두 데이터 전송이 적고 메모리가 필요하기 때문에 가속을 허용합니다. 동시에 int8 및 fp16 연산은 특수 하드웨어인 Tensor Core(Volta부터 시작하는 모든 GPU 아키텍처용)에서 실행될 수 있습니다. 
7. <b>기타:</b></br>
    - 빠른 C++ BeamSearch 구현
    - TensorParallelism 8 모드에 최적화된 올-리듀스. (모델의 가중치가 8개의 GPU로 분할)

# 3.2 NVIDIA FasterTransformer를 통한 전반적인 추론 파이프라인

<img src="./images/FT_pipeline.PNG">

## 3.3 Download and Build NVIDIA FasterTransformer library

In [1]:
!git clone https://github.com/NVIDIA/FasterTransformer.git
%cd FasterTransformer
!git checkout 6b3fd4392831f972d48127e881a048567dd92811

Cloning into 'FasterTransformer'...
remote: Enumerating objects: 3838, done.
remote: Counting objects: 100% (3408/3408), done.
remote: Compressing objects: 100% (2003/2003), done.
remote: Total 3838 (delta 1828), reused 2401 (delta 1373), pack-reused 430
Receiving objects: 100% (3838/3838), 20.18 MiB | 16.56 MiB/s, done.
Resolving deltas: 100% (1926/1926), done.
Checking out files: 100% (1046/1046), done.
/content/FasterTransformer
Note: checking out '6b3fd4392831f972d48127e881a048567dd92811'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at 6b3fd43 Update gpt_guide.md


### 라이브러리 구축하기

#### Install dependencies

We should install additional libraries like `cmake` for build automation, and we also should install compression libraries "zstd" and "libz-dev" that will allow decompressing weights of the GPT-J 6b model. </br>


In [2]:
!apt update
!DEBIAN_FRONTEND=noninteractive apt install -y cmake xz-utils zstd libz-dev

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://securit

#### Build library

We downloaded the FasterTransformer code from Github to build this library manually and use additional scripts in it that allow us to convert pre-trained files of the GPT-J into FT binary format to be used at the inference time.

Let's create a `build` directory that will be used as the main dir for the built library.

In [3]:
!mkdir -p build
%cd build 

/content/FasterTransformer/build


Let's build the library with the arg `-DBUILD_MULTI_GPU=ON ` that includes in the build pipeline multi-GPU cases like `GPT-J`.

In [4]:
!git submodule init && git submodule update
!cmake -DSM=xx -DCMAKE_BUILD_TYPE=Release -DBUILD_MULTI_GPU=ON ..
!make -j32

Submodule '3rdparty/Megatron-LM' (https://github.com/NVIDIA/Megatron-LM.git) registered for path '../3rdparty/Megatron-LM'
Submodule 'examples/tensorflow/bert/tensorflow_bert/bert' (https://github.com/google-research/bert.git) registered for path '../examples/tensorflow/bert/tensorflow_bert/bert'
Cloning into '/content/FasterTransformer/3rdparty/Megatron-LM'...
Cloning into '/content/FasterTransformer/examples/tensorflow/bert/tensorflow_bert/bert'...
Submodule path '../3rdparty/Megatron-LM': checked out '90e0a0dd08159e1c95f4f9d99bb8687f327d36c3'
Submodule path '../examples/tensorflow/bert/tensorflow_bert/bert': checked out 'eedf5716ce1268e56f0a50264a88cafad334ac61'
-- The CXX compiler identification is GNU 7.5.0
-- The CUDA compiler identification is NVIDIA 11.1.105
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Detecting 

Let's have a look at all pre-built binary files with examples and useful utilities.

In [5]:
print("All built examples:")
!ls ./bin

All built examples:
bert_example	   swin_example		 vit_example
bert_gemm	   swin_gemm		 vit_gemm
bert_int8_example  swin_int8_example	 vit_int8_example
decoding_gemm	   t5_gemm		 xlnet_correctness_example
gpt_example	   test_gemm		 xlnet_example
gpt_gemm	   test_logprob_kernels  xlnet_gemm
layernorm_test	   test_sampling


Looks like the FasterTransformer library was successfully built and ready for the inference of our models.

## 3.4 GPT-J 체크포인트 다운로드 및 구축하기

C 프로그래밍 언어로 GPT를 실행하려면 사용자가 텐서플로우 또는 파이토치의 체크포인트를 바이너리 파일로 변환한 다음 FasterTransformer C API로 로드해야 합니다. 유감스럽게도 OpenAI에서 출시한 공개 대형 모델은 없습니다. 하지만 현재 FasterTransformer는 다른 회사의 여러 샘플을 제공하고 있습니다. 첫 번째는 Megatron-LM(PyTorch에 의해 훈련됨)의 체크포인트입니다. 다른 선택사항은 OpenAI GPT-2 모델(TensorFlow에 의해 훈련됨)의 체크포인트를 활용하는 것입니다. 그리고 이전 예에서 우리가 사용했던 것과 같은 Eleuther.AI 의 GPT-J 6b를 사용할 수 있습니다.

가중치는 이미 다운로드해두었지만, 원하는 경우 언제든지 아래 줄에 대한 주석을 해제하여 다운로드할 수 있습니다.

In [6]:
# Go to the main course dir
%cd /dli/task

[Errno 2] No such file or directory: '/dli/task'
/content/FasterTransformer/build


In [7]:
# We already download weights for you to not to wait for them
# That's why these lines are commented
# !wget https://mystic.the-eye.eu/public/AI/GPT-J-6B/step_383500_slim.tar.zstd
!tar -axf ./weights/gpt-j/ft/step_383500_slim.tar.zstd -C ./model_repository/

tar: ./weights/gpt-j/ft/step_383500_slim.tar.zstd: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now


다운로드된 가중치를 살펴보겠습니다.

In [8]:
!ls ./model_repository/step_383500/

ls: cannot access './model_repository/step_383500/': No such file or directory


가중치는 여러 샤드로 분할됩니다. 동일한 방법으로 Megatron  프레임워크로 사전 훈련을 받았습다. 샤드는 훈련 과정에서 다른 장치에 올려진 가중치의 일부입니다.

이전 실습에서 사용했던 모델과 동일한 모델을 사용할 것이며, 토큰 처리 및 인코딩/디코딩을 위해 HuggingFace의 토크나이저를 사용할 것입니다. Huggingface에서 추론 단계에서 필요한 기본 Vocab 및 Merge 테이블 파일을 다운로드해야 합니다.

In [9]:
!wget https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-vocab.json -P models
!wget https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-merges.txt -P models

--2022-08-24 07:10:41--  https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-vocab.json
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.37.30
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.37.30|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1042301 (1018K) [application/json]
Saving to: ‘models/gpt2-vocab.json’

gpt2-vocab.json     100%[===================>]   1018K  --.-KB/s    in 0.03s   

2022-08-24 07:10:41 (31.3 MB/s) - ‘models/gpt2-vocab.json’ saved [1042301/1042301]

--2022-08-24 07:10:41--  https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-merges.txt
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.92.141
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.92.141|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 456318 (446K) [text/plain]
Saving to: ‘models/gpt2-merges.txt’

gpt2-merges.txt     100%[===================>] 445.62K  --.-KB/s    in 0.02s   

2022-08-24 07:10:41 (23.2 MB/s) 

## 3.5 추론을 위해 가중치를 FT 친화적인 포맷으로 변환

FasterTransformer의 C++ 백엔드에 대한 가중치는 이진 형식으로 변환되어야 합니다.</br>
FasterTransformer는 사전 훈련된 다양한 신경망을 위한 도구/스크립트를 제공합니. </br> 
GPT-J 가중치의 경우 이 `FasterTransformer/examples/pytorch/gptj/utils/gptj_ckpt_convert.py` 스크립트를 사용하여 체크포인트를 변환할 수 있습니다.</br> 
컨퍼터에는 다음 인자가 필요합니다. 

1. `--ckpt-dir`: megatron  모델의 경로
2.`--output-dir`: 변환된 모델의 출력 경로
3.`--n-inference-gpus`: 추론을 위한 텐서 병렬 크기

In [10]:
# get to directoy with build FasterTransformer library 
%cd FasterTransformer/build

[Errno 2] No such file or directory: 'FasterTransformer/build'
/content/FasterTransformer/build


In [11]:
# Converting weights for 1 GPU without parallelism
!python3 ../examples/pytorch/gptj/utils/gptj_ckpt_convert.py \
                                --output-dir ../../models/j6b_ckpt \
                                --ckpt-dir ../../model_repository/step_383500/ \
                                --n-inference-gpus 1

loading
loading shards for part 0
Traceback (most recent call last):
  File "../examples/pytorch/gptj/utils/gptj_ckpt_convert.py", line 276, in <module>
    checkpoint = main(in_path, num_layers)
  File "../examples/pytorch/gptj/utils/gptj_ckpt_convert.py", line 213, in main
    read_shard(f"{ckpt_dir}shard_{i}/", part) for i in range(total_shards)
  File "../examples/pytorch/gptj/utils/gptj_ckpt_convert.py", line 213, in <listcomp>
    read_shard(f"{ckpt_dir}shard_{i}/", part) for i in range(total_shards)
  File "../examples/pytorch/gptj/utils/gptj_ckpt_convert.py", line 73, in read_shard
    with open(file_path, "rb") as f:
FileNotFoundError: [Errno 2] No such file or directory: '../../model_repository/step_383500/shard_0/0.npz'


생성된 파일을 살펴보겠습니다. 우리는 조각난 가중치들을 추론에 사용하고자 하는 여러 GPU에 배치될 가중치 그룹으로 다시 결합했습니다. 지금은 GPU 1개를 사용하겠습니다.

In [12]:
!ls ../../models/j6b_ckpt/

ls: cannot access '../../models/j6b_ckpt/': No such file or directory


## 3.6 커널 자동 조정. 모델을 위한 가장 빠른 CUDA 커널 찾기## 

이제 커널 오토 튜닝을 해야 합니다. </br>
행렬 곱은 트랜스포머 기반 신경망에서 가장 중요하고 무거운 작업입니다. FT는 CuBLAS 및 CuTLASS 라이브러리의 기능을 사용하여 이러한 유형의 작업을 실행합니다. MatMul 연산은 "하드웨어" 수준에서 서로 다른 로우레벨 알고리즘을 사용하여 수십 가지 다른 방식으로 실행될 수 있다는 것을 아는 것이 중요합니다. GemmBatchedEx 함수는 MatMul 연산을 구현하고 “cublasGemmAlgo_t” 를 입력 매개 변수로 가지고 있습니다. 이 매개 변수를 사용하여, 우리는 연산을 위해 다른 로우 레벨의 알고리즘을 선택할 수 있습니다. FasterTransformer 라이브러리는 이 매개 변수를 사용하여 모든 로우 레벨 알고리즘의 실시간 벤치마크를 수행하고 입력 데이터에 대한 모델의 매개 변수(어텐션 레이어 크기, 어텐션 헤드 수, 히든 레이어 크기 등)에 가장 적합한 매개 변수를 선택합니다. 또한 FT는__expf(), __shfl_xor_sync()</br></br>와 같은 일부 네트워크 부분에 하드웨어 가속 로우 레벨 함수를 사용합니다.
FasterTransformer 라이브러리를 구축하는 단계(2단계)에서 작성된 `./FasterTransformer/build/bin/gpt_gemm`바이너리 파일을 실행해 보겠습니다. 이 파일은 모델의 매개 변수와 다음과 같은 추가 매개 변수를 입력으로 사용합니다. 
* `batch_size` 
* `beam_width` 
* `max_input_len` 
* `head_number` 
* `size_per_head`
* `inter_size`
* `vocab_size` 
* `data_type` 
* `tensor_para_size`

이 모든 매개 변수는 신경망의 가장 무거운 부분을 시뮬레이션하고 가짜 가중치와 다중 로우 레벨 알고리즘으로 이러한 부분의 연산을 실행하는 데 필요하며, 실제 추론 과정에서 나중에 몇 가지 가장 빠른 알고리즘을 찾습니다. 이러한 매개 변수는 수동으로 설정해야 합니다. GPT-J 모델의 경우 FasterTransformer 팀은`../examples/cpp/gptj/gptj_config.ini` 구성 파일을 준비했습니다. 여기에는 GPT-J 모델에 대한 정보와 향후 추론에 대한 정보가 포함된 매개 변수가 포함되어 있습니다. </br>
<b>이러한 모든 테스트는 나중에 추론을 위해 사용될 GPU에서 실행되어야 합니다.

In [13]:
!CUDA_VISIBLE_DEVICES=1  ./bin/gpt_gemm 1 1 128 16 256 16384 50256 1 1

[INFO] arguments: 
  batch_size: 1 
  beam_width: 1 
  max_input_len: 128 
  head_num: 16 
  size_per_head: 256 
  inter_size: 16384 
  vocab_size: 50256 
  data_type: 1 
  tensor_para_size: 1 

terminate called after throwing an instance of 'std::runtime_error'
  what():  [FT][ERROR] CUDA runtime error: no CUDA-capable device is detected /content/FasterTransformer/src/fastertransformer/models/multi_gpu_gpt/gpt_gemm.cc:69 



그 결과, 시도한 알고리즘과 각 알고리즘에 대한 실행 시간에 대한 정보가 들어 있는  `gemm_config.in` 파일을 디렉터리에 받았습니다.  이 구성 파일은 나중에 바인딩 또는 C++ 코드를 통한 추론 예제에서 FT 라이브러리에서 사용됩니다.

# 3.6 C++ 바인딩을 사용한 GPT-J 추론

## 3.6.1 1개의 GPU에서 추론하기 

시작하기 전에 이전 과정에서 활용된`./FasterTransformer/examples/cpp/gptj/gptj_config.ini` 구성 파일을 한 번 더 살펴봐야 합니다.  여기에는 GPT 디코더 레이어 수, 헤드 수, MLP 레이어의 히든 뉴런 수 등과 같은 GPT-J 모델에 대한 주요 정보가 포함되어 있습니다. 동시에 배치 크기, 온도, top_k, top_p, beam_search 매개 변수 및 정밀도 모드(FP16 또는 FP32)와 같은 C++에서의 추론을 위한 하이퍼 매개 변수를 포함합니다.

`./FasterTransformer/examples/cpp/gptj/gptj_config.ini`를 구성하여 GPT-J C++ 추론 사례를 실행합니다.</br>
우리는 이 파일을 JupyterLab에서 열고 HuggingFace에서 했던 추론 파이프라인과 유사한 추론 사례를 만들기 위해 이 라인들을 업데이트해야 합니다. </br>
<b>`./FasterTransformer/examples/cpp/gptj/gptj_config.ini`를 주피터 노트북에서 엽니다. 다음 행을 변경합니다. </b>
* max_batch_size=<b>1</b> # HF 파이프라인과 유사하게 설정
* top_k=<b>50</b> # HF 파이프라인과 유사하게 설정
* top_p=<b>1</b> # HF 파이프라인과 유사하게 설정
* is_half=<b>1</b> #FP16 모드에서 모델 실행을 위해 1 로 설정(반정밀도), 아니면, GPU 가용 메모리가 충분치 않아 OOM 에러 발생
* tensor_para_size=<b>1</b> #텐서 병렬화 모드 없이 1개 GPU에서 구동
* model_dir=<b>../../models/j6b_ckpt/</b> # pre-baked 가중치 경로
* request_batch_size=<b>1</b>
* request_output_len=<b>120</b> # 출력길이는 HF와 유사하게 설정

다른 모든 매개 변수는 파일에 표시된 것과 같아야 합니다.

사전 빌드된 스크립트 `./FasterTransformer/build/bin/gptj_example`을 실행해 보겠습니다. `./FasterTransformer/examples/cpp/gptj/gptj_example.cpp`C++ 파일에서 구축했습니다. </br>
이 파일은 `./FasterTransformer/examples/cpp/gptj/gptj_config.ini`로부터 우리가 준비한 가중치와 구성을 적용합니다. 이 바이너리 파일은 `./FasterTransformer/examples/cpp/gptj/start_ids.csv` 로부터 입력 토큰을 프롬프트로 가져와 GPT-J의 작업 과정에서 생성된 토큰을 포함하는 `out`  텍스트 파일을 생성합니다.

`CUDA_VISIBLE_DEVICES` 환경 변수를 사용하면 현재 태스크/프로세스/스크립트에 사용할 GPU를 선택할 수 있습니다. 시스템에 4개의 GPU가 있으며(0-3 숫자로 구성, 0부터 시작) 이 변수를 사용하여 GPU 1 (2번째 GPU)에서 현재 예제를 실행할 것입니다. 이전 작업(2번째 노트북)이 GPU 0에서 시작되었고 Jupyter 커널을 중지하지 않았다면 모델이 이미 존재하기 때문에 GPU 1을 사용할 것입니다. 그래서 우리는 OOM 오류를 피할 수 있습니다.

In [ ]:
!CUDA_VISIBLE_DEVICES=1 ./bin/gptj_example

우리는 길이가 같은 문장의 추론 시간이 <b>1.8 초</b>, 즉 기본 PyTorch + HuggingFace 파이프라인보다<b>x3.5</b> 배 더 빠름을 알 수 있습니다.</br>
`out`파일을 디코드하여 신경망에서 무엇이 생성되었는지 알아보겠습니다.

In [ ]:
!python3 ../examples/pytorch/gpt/utils/gpt_token_converter.py \
                       --vocab_file=../../models/gpt2-vocab.json  \
                       --bpe_file=../../models/gpt2-merges.txt

사용자들은`examples/cpp/multi_gpu_gpt/gpt_config.ini`에서 구성 인자들의 세부 정보를 볼 수 있습니다. 모델 경로, 모델 크기, 텐서 병렬 크기 및 일부 하이퍼 파라미터를 제어합니다.

## 3.6.2 2개의 GPU에서 추론하기

FasterTransformer에 통합된 텐서-병렬화 기술을 사용하여 동일한 추론 단계를 실행하지만 거의 두 배 더 빠르게 실행할 수 있습니다.

텐서-병렬화 기법에 대해 좀 더 자세히 알아보겠습니다.
아래 그림에서 텐서-병렬화 기법의 기본 예를 볼 수 있습니다. 많은 신경망의 기본 연산인 단순 행렬 곱(MatMul) 연산은 여러 가지 방법으로 계산할 수 있습니다. 일부 방법에서는 행렬을 여러 조각으로 분할하고 여러 장치(GPU)에서 MatMul 연산의 일부를 병렬로 계산한 다음 결과를 집계할 수 있습니다. 즉, 2개의 GPU를 사용할 수 있고 그 사이에 빠르게 상호 연결할 수 있는 경우(NVLink/NVSwitch), 최대 2배 낮은 대기 시간으로 동일한 MatMul 연산을 병렬로 계산할 수 있습니다.

<img src="./images/parallelism-tp-parallel_gemm.png" style="width: 800px;">

마지막 사진은 Hugging Face 사이트에서 찍은 사진입니다. https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/parallelism-tp-parallel_gemm.png

동일한 기술을 사용하여 현재 Transformer 모델의 주요 블록인 Attention과 MLP 블록을 병렬화할 수 있습니다:
<img src="./images/parallelism-tp-parallel_self_attention.png" style="width: 600px;"></br>
<img src="./images/parallelism-tp-parallel_shard_processing.png" style="width: 600px;">

텐서-병렬화=2 모드에서 GPT-J의 추론을 실행해 보겠습니다. 즉, 가중치를 두 부분으로 나누어 NVLink와 연결된 여러 GPU에 배치해야 합니다. FasterTransformer에는 스크립트에 특별한 인자들을 추가하였으며 우리는 아무것도 구현할 필요가 없습니다.

가중치 준비에 대해서는 동일한 스크립트를 사용하되 TP=2 모드에 대한 가중치를 준비하기 위해`--n-inference-gpus`를 `2` 로 설정합니다.

In [ ]:
# Converting weights for 2 GPU without parallelism
!python3 ../examples/pytorch/gptj/utils/gptj_ckpt_convert.py \
                                --output-dir ../../models/j6b_ckpt \
                                --ckpt-dir ../../model_repository/step_383500/ \
                                --n-inference-gpus 2

생성된 파일을 살펴보겠습니다. 우리는 조각난 가중치를 추론에 사용하고자 하는 다수의 GPU에 배치될 가중치 그룹으로 다시 결합했습니다. 일단 GPU 2개를 사용하겠습니다.

In [ ]:
!ls ../../models/j6b_ckpt/

이제 커널 오토 튜닝을 다시 실행하여 2xGPU 모드에서 가장 빠른 커널을 찾아야 합니다. 먼저 이전 단계에서 이미 1xGPU 모드에 대해 찾은 커널로 추론을 실행해 보겠습니다. 우리는 이 비최적화된 추론을 나중에 커널오토튜닝과 비교하여 이 작업이 제공하는 가속도를 보기 위해 수행합니다.

In [ ]:
!CUDA_VISIBLE_DEVICES=1,2 mpirun -n 2 --allow-run-as-root ./bin/gptj_example

최적 커널이 아닌 경우의 추론은 <b>~1.4초<b>입니다. 우리는 1xGPU 모드와 비교하여 가속을 달성했지만, 더 나은 결과를 낼 수 있습니다. 이제 다른 알고리즘이 더 잘 작동할 가능성이 있기 때문에 Tp 2 체제에 대한 커널 오토 튜닝을 수행해야 합니다. 신경망에 대해 동일한 매개 변수를 사용하지만, 이때`tensor_para_size = 2`로 설정해야 합니다.

이전 작업(노트북 02)이 GPU 0에서 시작되었고 Jupyter 커널을 중지하지 않았다면 모델이 이미 있기 때문에 GPU 1을 사용할 것입니다. (OOM 오류를 피하기 위함입니다.)

In [ ]:
!CUDA_VISIBLE_DEVICES=1  ./bin/gpt_gemm 1 1 128 16 256 16384 50256 1 2

첫 번째 랩에서 서버에서 4개의 GPU를 사용할 수 있다는 것을 기억합니다. 그 중 2개를 사용하여 TP=2 체제에서 추론을 시작하겠습니다.

GPT-J C++ 추론 사례를 실행하기 위해 `./FasterTransformer/examples/cpp/gptj/gptj_config.ini`를 구성합니다.</br>
우리는 이 파일을 JupyterLab에서 열고 이 라인들을 업데이트하여 이전에 GPU가 2개인 추론 파이프라인과 유사한 추론 사례를 만들어야 합니다. </br>
* tensor_para_size=<b>2</b> #텐서 병렬화 모드로 2xGPUs에서 실행

다른 모든 파라미터는 FT를 사용한 이전 실행과 동일해야 합니다.
 
이전 작업(노트북 02)이 GPU 0에서 시작되었고 Jupyter 커널을 중지하지 않았다면 모델이 이미 있기 때문에 GPU 2와 3(번호 1과 2)을 사용할 것입니다. OOM 오류를 피하기 위해서이며, 이제 최적화된 커널로 추론을 실행해 보겠습니다.

In [ ]:
!CUDA_VISIBLE_DEVICES=1,2 mpirun -n 2 --allow-run-as-root ./bin/gptj_example

동일한 길이의 문장의 추론 시간이 기본 PyTorch + HuggingFace 파이프라인보다 <b>x5</b>배 빠른 <b>1.25초 </b>임을 알 수 있습니다! 물론 여기서 2개의 GPU를 사용했지만, 이는 추론에 GPU를 더 추가하여 대기 시간을 단축할 수 있는 방법을 보여줍니다.</br>
`out` 파일을 디코딩하여 신경망에서 무엇이 생성되었는지 알아보겠습니다.

In [ ]:
!python3 ../examples/pytorch/gpt/utils/gpt_token_converter.py \
                               --vocab_file=../../models/gpt2-vocab.json  \
                               --bpe_file=../../models/gpt2-merges.txt

## 결과

결과들을 비교해 보겠습니다! 우리가 볼 수 있듯이, 우리는 HuggingFace 와 텐서-병렬화 2 체제에서 <b>5x/b> 배 가속을 얻었으며 TP=1 모드에서 FasterTransformer를 사용하여 3.5배 가속도를 얻습니다. 훌륭한 성능을 확인할 수 있었습니다! 이번 C++ 예제와 PyTorch/TF 바인딩을 활용하여 복잡한 추론 파이프라인에서 적용할 수 있습니다. </br> 
동시에, 보다 통합된 추론 엔진/서빙 솔루션을 원하신다면 다음 노트북에서 살펴보도록 하겠습니다.

<h2 style="color:green;">축하합니다!</h2>

[Triton Inference Server와 FasterTransformer를 백엔드로 하는 GPT-J 6b 모델의 추론](04_FTAndTritonRunRemoteInferenceOfTheGPT-J.ipynb)으로 이동하십시오.